In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [2]:
from langchain.tools import tool

@tool # subagent 1 - calculates square root
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool # subagent 2 - calculates square
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [3]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model='gpt-5-nano',
    tools=[square_root]
)

subagent_2 = create_agent(
    model='gpt-5-nano',
    tools=[square]
)

## Calling subagents

In [4]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number""" # tells main agent what subagent 1 does
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number""" # tells main agent what subagent 2 does
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model='gpt-5-nano',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [5]:
question = "What is the square root of 625?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 625?', additional_kwargs={}, response_metadata={}, id='74ec6125-42c5-433a-ba84-94388b4d7159'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 410, 'prompt_tokens': 202, 'total_tokens': 612, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D90jM4NI6Lm1JV0mwi4mrfNPPGxKl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c5a37-cbb0-7432-976a-70e143c66e4a-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 625}, 'id': 'call_0H1RA2ZtYtgSFetggWJS7zXw', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 202, 'o

In [7]:
question = "What is the square of 10?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [8]:
pprint(response)

{'messages': [HumanMessage(content='What is the square of 10?', additional_kwargs={}, response_metadata={}, id='77364838-a90f-440c-b300-fbc1ad12eea5'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 284, 'prompt_tokens': 201, 'total_tokens': 485, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D90k2r9TCGIooTSVU5LJJgYzaJXDe', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c5a38-71a6-75f1-9064-4b4ea2d78ddf-0', tool_calls=[{'name': 'call_subagent_2', 'args': {'x': 10}, 'id': 'call_0U2umWuc7cfNE7MqSUdZTxtE', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 201, 'output_t

In [10]:
question = "What is 10 squared?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

pprint(response)

{'messages': [HumanMessage(content='What is 10 squared?', additional_kwargs={}, response_metadata={}, id='bd164c99-1967-4c98-b7c9-1826a42b7d33'),
              AIMessage(content='100. (10 squared means 10 × 10 = 100.)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 199, 'total_tokens': 351, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D90kquXkCwn1IP6RtOX7wqZpMfuab', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c5a39-32a0-7600-81b8-a107c76eae5c-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 199, 'output_tokens': 152, 'total_tokens': 351, 'input_token_details': {'audio': 0, 'cache_rea

Run Langsmith Trace on `.py` file to ensure proper subagent is being called